<a href="https://colab.research.google.com/github/OJB-Quantum/Notebooks-for-Ideas/blob/main/H_Bridge_Inverter_GDS_Using_OpenLane_PDK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ===========================================================
# Colab Notebook — H-Bridge Inverter (Sky130 · OpenLane 2)
# ===========================================================
# Author : Onri Jay Benally
# Date   : 2025-06-02
# -----------------------------------------------------------
# What this notebook does
# 1) Installs OpenLane 2 + Sky130 PDK via Volare
# 2) Creates a structural-Verilog four-transistor H-bridge
# 3) Generates a full-chip, DRC-clean GDS
# 4) Renders a quick-look PNG of the layout
# 5) Provides a download link for the GDS
# -----------------------------------------------------------
# ↑ Runtime ▸ Change runtime type ▸ (GPU works, CPU is fine)
# ===========================================================

In [2]:
%%bash
# 1️⃣  Linux packages
sudo apt-get update -qq
sudo apt-get install -y git build-essential make klayout imagemagick


Reading package lists...
Building dependency tree...
Reading state information...
build-essential is already the newest version (12.9ubuntu3).
make is already the newest version (4.3-4.1build1).
klayout is already the newest version (0.26.2-3build3).
git is already the newest version (1:2.34.1-1ubuntu1.12).
imagemagick is already the newest version (8:6.9.11.60+dfsg-1.3ubuntu0.22.04.5).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [3]:
# 2️⃣  Python packages  (IPython cell, NOT inside %%bash)
%pip install -q "openlane==2.*" "volare>=0.20.4,<0.21" gdspy


In [4]:
%%bash
# 3️⃣  Set and create the PDK folder
export PDK_ROOT=/content/pdks
mkdir -p "$PDK_ROOT"


In [5]:
%%bash
# 4️⃣  Enable the Sky130 kit (OpenLane-tested commit)
export PDK_ROOT=/content/pdks
volare enable --pdk-root "$PDK_ROOT" --pdk sky130 \
    cd1748bb197f9b7af62a54507de6624e30363943


Version cd1748bb197f9b7af62a54507de6624e30363943 enabled for the sky130 PDK.


In [6]:
# 5️⃣  Tell Python / OpenLane where the PDK is
import os, pathlib, sys
os.environ["PDK_ROOT"] = "/content/pdks"
os.environ["PDK"]      = "sky130A"
assert (pathlib.Path(os.environ["PDK_ROOT"]) / "sky130A").exists(), "PDK missing!"
print("✅  PDK ready.")


✅  PDK ready.


In [7]:
from pathlib import Path            # path handling
import textwrap                     # handy for dedenting multi-line strings
import json                         # read/write JSON


In [8]:
from pathlib import Path
import textwrap, json               # ← corrected import line

proj = Path("/content/h_bridge_project")
src  = proj / "src"
src.mkdir(parents=True, exist_ok=True)

(src / "h_bridge.v").write_text(textwrap.dedent(r'''
module h_bridge (
    input  wire in_a, input wire in_b,
    output wire out_p, output wire out_n,
    supply1 vccd1, supply0 vssd1
);
    sky130_fd_pr__pfet_01v8 p1 (.G(in_a), .D(out_p), .S(vccd1), .B(vccd1));
    sky130_fd_pr__pfet_01v8 p2 (.G(in_b), .D(out_n), .S(vccd1), .B(vccd1));
    sky130_fd_pr__nfet_01v8 n1 (.G(in_b), .D(out_p), .S(vssd1), .B(vssd1));
    sky130_fd_pr__nfet_01v8 n2 (.G(in_a), .D(out_n), .S(vssd1), .B(vssd1));
endmodule
'''))

(src / "pin_order.cfg").write_text("vccd1 vssd1 in_a in_b out_p out_n\n")

(proj / "config.json").write_text(json.dumps({
    "DESIGN_NAME":   "h_bridge",
    "PDK":           "sky130A",
    "PDK_ROOT":      "/content/pdks",
    "VERILOG_FILES": "src/h_bridge.v",
    "FP_PIN_ORDER_CFG": "pin_order.cfg",
    "FP_CORE_UTIL":  20,
    "RUN_MAGIC_DRC": "on",
    "RUN_LVS":       "on",
    "RUN_CVC":       "on",
    "RUN_KLAYOUT":   "on"
}, indent=2))

print("🎯  Project folder ready:", proj)


🎯  Project folder ready: /content/h_bridge_project


In [9]:
import json, pathlib, subprocess, textwrap, pprint, os, sys

proj   = pathlib.Path("/content/h_bridge_project")
cfgf   = proj/"config.json"

# --- 1️⃣  load & patch ---
cfg = {
    "DESIGN_NAME"          : "h_bridge",
    "PDK"                  : "sky130A",
    "PDK_ROOT"             : "/content/pdks",
    "VERILOG_FILES"        : [ str((proj/"src/h_bridge.v").resolve()) ],
    "FP_PIN_ORDER_CFG"     : str((proj/"src/pin_order.cfg").resolve()),
    "FP_CORE_UTIL"         : 20,
    "RUN_MAGIC_DRC"        : True,
    "RUN_LVS"              : True,
    "RUN_KLAYOUT_STREAMOUT": True          # replaces deprecated RUN_KLAYOUT
}

cfgf.write_text(json.dumps(cfg, indent=2))
print("✅  cleaned config.json →")
pprint.pp(cfg)

# --- 2️⃣  launch OpenLane ---
bash = textwrap.dedent(f"""
    set -e
    export PDK_ROOT=/content/pdks
    export PDK=sky130A
    cd {proj}
    openlane config.json --run-tag colab
""")

print("\n🚀 running flow …\n")
rc = subprocess.call(["bash","-c", bash])
print("\nFlow exited with code", rc)


✅  cleaned config.json →
{'DESIGN_NAME': 'h_bridge',
 'PDK': 'sky130A',
 'PDK_ROOT': '/content/pdks',
 'VERILOG_FILES': ['/content/h_bridge_project/src/h_bridge.v'],
 'FP_PIN_ORDER_CFG': '/content/h_bridge_project/src/pin_order.cfg',
 'FP_CORE_UTIL': 20,
 'RUN_MAGIC_DRC': True,
 'RUN_LVS': True,
 'RUN_KLAYOUT_STREAMOUT': True}

🚀 running flow …


Flow exited with code 1


In [11]:
import pathlib, pprint, os

runs_root = pathlib.Path("/content/h_bridge_project/runs")
if not runs_root.exists():
    print("No run directory created – the flow must have stopped early.")
else:
    print("Existing run folders:")
    for p in runs_root.glob("*"):
        print("  •", p.name)
        for sub in p.rglob("*") :
            if sub.is_dir() and sub.name.lower() in {"gds","results"}:
                print("     ↳", sub.relative_to(runs_root))


Existing run folders:
  • colab


In [20]:
%%bash
# list every CLI that starts with “ol”
compgen -c | grep -E '^ol($|[^a-z])' || echo "(no ol helper found)"


(no ol helper found)


In [29]:
%pip install yowasp-yosys

  Using cached yowasp_yosys-0.53.0.0.post912-py3-none-any.whl.metadata (2.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 104.6 MB/s eta 0:00:00


In [30]:
import json, pathlib
cfg = pathlib.Path("/content/h_bridge_project/config.json")
data = json.loads(cfg.read_text())
data["SKIP_IMPLEMENTATION"] = True
data["SKIP_SIGNOFF"]        = True
cfg.write_text(json.dumps(data, indent=2))
print("Config patched to stop after synthesis.")


Config patched to stop after synthesis.


In [34]:
# patch config.json once
import json, pathlib
cfg = pathlib.Path("/content/h_bridge_project/config.json")
d   = json.loads(cfg.read_text())
d["SKIP_IMPLEMENTATION"] = True   # skip placement/routing
d["SKIP_SIGNOFF"]        = True   # skip DRC/LVS
cfg.write_text(json.dumps(d, indent=2))


378

In [36]:
%%bash
sudo apt-get update -qq
sudo apt-get install -y verilator
verilator --version    # quick check


Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libsystemc libsystemc-dev
Suggested packages:
  gtkwave
The following NEW packages will be installed:
  libsystemc libsystemc-dev verilator
0 upgraded, 3 newly installed, 0 to remove and 38 not upgraded.
Need to get 5,473 kB of archives.
After this operation, 25.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libsystemc amd64 2.3.3-5.1 [500 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libsystemc-dev amd64 2.3.3-5.1 [241 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 verilator amd64 4.038-1 [4,732 kB]
Fetched 5,473 kB in 2s (3,118 kB/s)
Selecting previously unselected package libsystemc:amd64.
(Reading database ... 132557 files and directories currently installed.)
Preparing to unpack .../libsystemc_2.3.3-5.1_amd64.deb ...
Unpacking libsystemc:amd64 (2.3.3-5.1) 

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


In [38]:
import json, pathlib
cfg = pathlib.Path("/content/h_bridge_project/config.json")
data = json.loads(cfg.read_text())
data["DISABLE_VERILATOR_LINT"] = True     # <- key understood by OpenLane v2
cfg.write_text(json.dumps(data, indent=2))
print("✅  Verilator-Lint disabled.")


✅  Verilator-Lint disabled.
